# Define

## import

In [1]:
import torch #基本モジュール
from torch.autograd import Variable #自動微分用
import torch.nn as nn #ネットワーク構築用
import torch.optim as optim #最適化関数
import torch.nn.functional as F #ネットワーク用の様々な関数
import torch.utils.data #データセット読み込み関連
import torchvision #画像関連
from torch import Tensor
from torchvision import datasets, models, transforms #画像用データセット諸々

import numpy as np
import argparse
import json
from logging.config import dictConfig
from logging import getLogger
import os
import time
from google.colab import files
import itertools

## Components

In [2]:
class Zero(nn.Module):
  def __init__(self, stride):
    super(Zero, self).__init__()
    self.stride = stride

  def forward(self, x):
    if self.stride == 1:
      return x.mul(0.)
    return x[:,:,::self.stride,::self.stride].mul(0.)

In [3]:
class FactorizedReduce(nn.Module):
  def __init__(self, C_in, C_out, affine=True):
    super(FactorizedReduce, self).__init__()
    assert C_out % 2 == 0
    self.relu = nn.ReLU(inplace=False)
    self.conv_1 = nn.Conv2d(C_in, C_out // 2, 1, stride=2, padding=0, bias=False)
    self.conv_2 = nn.Conv2d(C_in, C_out // 2, 1, stride=2, padding=0, bias=False) 
    self.bn = nn.BatchNorm2d(C_out, affine=affine)

  def forward(self, x):
    x = self.relu(x)
    # strideの偶奇による情報ロスを防ぐ
    out = torch.cat([self.conv_1(x), self.conv_2(x[:,:,1:,1:])], dim=1)
    out = self.bn(out)
    return out

In [4]:
class ReLUConvBN(nn.Module):

  def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
    super(ReLUConvBN, self).__init__()
    self.op = nn.Sequential(
      nn.ReLU(inplace=False),
      nn.Conv2d(C_in, C_out, kernel_size, stride=stride, padding=padding, bias=False),
      nn.BatchNorm2d(C_out, affine=affine)
    )

  def forward(self, x):
    return self.op(x)

In [5]:
class DilConv(nn.Module):
    
  def __init__(self, C_in, C_out, kernel_size, stride, padding, dilation, affine=True):
    super(DilConv, self).__init__()
    self.op = nn.Sequential(
      nn.ReLU(inplace=False),
      nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=C_in, bias=False),
      nn.Conv2d(C_in, C_out, kernel_size=1, padding=0, bias=False),
      nn.BatchNorm2d(C_out, affine=affine),
      )

  def forward(self, x):
    return self.op(x)

In [6]:
class SepConv(nn.Module):
    
  def __init__(self, C_in, C_out, kernel_size, stride, padding, affine=True):
    super(SepConv, self).__init__()
    self.op = nn.Sequential(
      nn.ReLU(inplace=False),
      nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, padding=padding, groups=C_in, bias=False),
      nn.Conv2d(C_in, C_in, kernel_size=1, padding=0, bias=False),
      nn.BatchNorm2d(C_in, affine=affine),
      nn.ReLU(inplace=False),
      nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=1, padding=padding, groups=C_in, bias=False),
      nn.Conv2d(C_in, C_out, kernel_size=1, padding=0, bias=False),
      nn.BatchNorm2d(C_out, affine=affine),
      )

  def forward(self, x):
    return self.op(x)

In [7]:
class Edge(nn.Module):
  def __init__(self, operators, theta=None):
    super(Edge, self).__init__()
    self.operators = operators

  def forward(self, input: Tensor, theta: Tensor) -> Tensor:
    return sum(t * op(input) for t, op in zip(theta, self.operators))

In [8]:
class Graph():
  def __init__(self, node_num : int, input : int=1, output : int=1):
    self.node_num = node_num
    self.input = input
    self.output = output
    self.middle = node_num - input - output
    self._graph = self._make_graph(node_num)
    self._order = self._ordered_edges(1)
  
  def edges(self):
    return self._graph

  def _ordered_edges(self, agg : int):
    g = self._graph
    r = [i for i in range(self.node_num)][self.input:-self.output]
    return [[idx for idx, t in enumerate(g) if t[agg] == i] for i in r]

  # edge indices to each intermediate node
  def ordered_edges(self):
    return self._order

  def size(self):
    return len(self._graph)

  def _make_graph(self, num : int):
    l = [i for i in range(num - self.output)]
    return [(s, e) for (s, e) in itertools.combinations(l, 2) if e >= self.input]

In [9]:
import sys
from graphviz import Digraph

def plot(graph, theta, multi, filename):
  g = Digraph(
      format='png', #pdf
      edge_attr=dict(fontsize='20', fontname="times"),
      node_attr=dict(style='filled', shape='rect', align='center', fontsize='20', height='0.5', width='0.5', penwidth='2', fontname="times"),
      engine='dot')
  g.body.extend(['rankdir=LR'])

  for i in range(graph.node_num):
    color = 'lightblue'
    color = 'darkseagreen2' if graph.input > i else color
    color = 'palegoldenrod' if graph.node_num - graph.output <= i else color
    g.node(str(i), fillcolor=color)
  
  for idx, (s, e) in enumerate(graph.edges()):
    op = [CANDIDATE[i] for i, p in enumerate(theta[idx]) if p >= 1.0]
    if len(op) == 0: continue
    g.edge(str(s), str(e), label=op[0], fillcolor="gray")

  for i in range(graph.node_num - 1 - multi, graph.node_num - 1):
    g.edge(str(i), str(graph.node_num - graph.output), fillcolor="gray")

  g.render(filename, view=True)
  return g

## Operator

In [10]:
OPS = {
  'none' : lambda C, stride, affine: Zero(stride),
  'skip_connect' : lambda C, stride, affine: nn.Identity() if stride == 1 else FactorizedReduce(C, C, affine=affine),
  'avg_pool_3x3' : lambda C, stride, affine: nn.Sequential(nn.AvgPool2d(3, stride=stride, padding=1, count_include_pad=False), nn.BatchNorm2d(C, affine=False)),
  'max_pool_3x3' : lambda C, stride, affine: nn.Sequential(nn.MaxPool2d(3, stride=stride, padding=1), nn.BatchNorm2d(C, affine=False)),
  'conv_3x3' : lambda C, stride, affine: nn.Conv2d(C, C, 3, stride=stride, padding=1),
  'conv_5x5' : lambda C, stride, affine: nn.Conv2d(C, C, 5, stride=stride, padding=2),
  'ReLUConvBN' : lambda C, stride, affine: ReLUConvBN(C, C, 3, stride=stride, padding=1),
  'sep_conv_3x3' : lambda C, stride, affine: SepConv(C, C, 3, stride, 1, affine=affine),
  'sep_conv_5x5' : lambda C, stride, affine: SepConv(C, C, 5, stride, 2, affine=affine),
  'sep_conv_7x7' : lambda C, stride, affine: SepConv(C, C, 7, stride, 3, affine=affine),
  'dil_conv_3x3' : lambda C, stride, affine: DilConv(C, C, 3, stride, 2, 2, affine=affine),
  'dil_conv_5x5' : lambda C, stride, affine: DilConv(C, C, 5, stride, 4, 2, affine=affine),
  'conv_7x1_1x7' : lambda C, stride, affine: nn.Sequential(
    nn.ReLU(inplace=False),
    nn.Conv2d(C, C, (1,7), stride=(1, stride), padding=(0, 3), bias=False),
    nn.Conv2d(C, C, (7,1), stride=(stride, 1), padding=(3, 0), bias=False),
    nn.BatchNorm2d(C, affine=affine)
    ),
}

In [11]:
CANDIDATE = [
  'conv_3x3',
  'conv_5x5',
  # 'ReLUConvBN',
  'avg_pool_3x3',
  'max_pool_3x3',
  'skip_connect',
  'none',
]
CANDIDATE = [
  'none',
  'skip_connect',
  # 'avg_pool_3x3',
  'max_pool_3x3',
  'sep_conv_3x3',
  'sep_conv_5x5',
  # 'sep_conv_7x7',
  'dil_conv_3x3',
  'dil_conv_5x5',
  # 'conv_7x1_1x7',
]

## module

In [12]:
from typing import List
class Cell(nn.Module):
  def __init__(self, names, graph, cs, multi, reduces=[False], search=True):
    super(Cell, self).__init__()
    assert len(cs) == len(reduces) + 1
    c = cs[-1]
    self.reduce = reduces[-1]
    self.pres = nn.ModuleList([
      FactorizedReduce(c_prev, c, affine=False) if reduce else
      ReLUConvBN(c_prev, c, 1, 1, 0, affine=False)
      for c_prev, reduce in zip(cs[:-1], reduces[:-1] + [False])
    ])

    self.search = search
    self.graph = graph
    self.multi = multi
    self.edges = nn.ModuleList(
      [Edge(self._mix_operators(names, c, r)) for r in self.graph.edges()]
    )

  def _mix_operators(self, names, c, r):
    modules = []
    stride = 2 if self.reduce and r[0] < self.graph.input else 1
    affine = not self.search
    for name in names:
      modules += [OPS[name](c, stride, affine)]
    return nn.ModuleList(modules)

  def forward(self, inputs: List[Tensor], theta: Tensor) -> Tensor:
    nodes = [p(i) for p, i in zip(self.pres, inputs)]

    refs = self.graph.edges()
    for idc in self.graph.ordered_edges():
      output = sum(self.edges[idx](nodes[refs[idx][0]], theta[idx]) for idx in idc)
      nodes += [output]
    
    return torch.cat(nodes[-self.multi:], dim=1)

In [13]:

# TODO : check point (epoch)
# TODO : show layer shape
# TODO : replace to networkx
class CellNetwork(nn.Module):
  def __init__(self, depth = 4, node_num = 4, input = 1, init_channel = 16, class_num = 10, multi = -1, search = True):
    super(CellNetwork, self).__init__()
    self.depth = depth
    self.search = search
    self.onehot = False
    self.graph = Graph(node_num, input)
    self.multi = self.graph.middle if multi <= 0 else multi
    self.init_modules(3, self.multi, class_num, init_channel)
    self.init_theta()
    
    print(self.graph.edges())
    print(CANDIDATE)

  def is_reduce(self, idx):
    return self.depth//3 == idx or 2*self.depth//3 == idx

  def init_theta(self, zero=True, delta=1e-3):
    if zero:
      normal_theta = torch.zeros(self.graph.size(), len(CANDIDATE), requires_grad=True)
      reduce_theta = torch.zeros(self.graph.size(), len(CANDIDATE), requires_grad=True)
    else:
      normal_theta = delta * torch.randn(self.graph.size(), len(CANDIDATE), requires_grad=True)
      reduce_theta = delta * torch.randn(self.graph.size(), len(CANDIDATE), requires_grad=True)
    self.thetas = [normal_theta, reduce_theta]

  def init_modules(self, c, multi, class_num, ini_c):
    # stem
    # c_n = c * multi
    c_n = ini_c
    self.stem = nn.Sequential(
      nn.Conv2d(c, c_n, 3, padding=1, bias=False),
      nn.BatchNorm2d(c_n)
    )
    c = ini_c

    # cells
    input = self.graph.input
    reduces = [False for _ in range(input)]
    cs = [c_n for _ in range(input)]
    self.cells = nn.ModuleList()
    for i in range(self.depth):
      reduces += [self.is_reduce(i)]
      if self.is_reduce(i):
        c *= 2
      
      cell = Cell(CANDIDATE, self.graph, cs[-input:] + [c], multi, reduces=reduces[-input:], search=self.search)
      self.cells += [cell]
      cs += [multi * c]
    
    # classify
    self.pooling = nn.AdaptiveAvgPool2d(1)
    self.linear = nn.Linear(cs[-1], class_num)

  def forward(self, input) -> Tensor:
    s = self.stem(input)
    input_num = self.graph.input
    stas = [s] * input_num

    for idx, cell in enumerate(self.cells):
      theta = self.thetas[1] if cell.reduce else self.thetas[0]
      weights = theta if self.onehot else F.softmax(theta, dim=-1)
      s = cell(stas[-input_num:], weights)
      stas += [s]

    out = self.pooling(s)
    return self.linear(out.view(out.size(0), -1))
  
  def to(self, *args, **kwargs):
    with torch.no_grad():
      for idx, theta in enumerate(self.thetas):
        self.thetas[idx] = theta.to(*args, **kwargs)

    return super(CellNetwork, self).to(*args, **kwargs)

  def learn_theta(self, is_learn: bool):
    for theta in self.thetas:
      theta.requires_grad = is_learn

  def sampling(self, inplace=True):
    def _sampling(theta, degree=2, graph=self.graph, ignore=CANDIDATE.index('none')):
      with torch.no_grad():
        for t in theta:
          t[ignore] = t.min(0).values
          max = t.max(0)
          t[:] = 0.0
          t[max.indices] = max.values

        for edges in graph.ordered_edges():
          values = [(e, theta[e].argmax(), theta[e].max(0).values) for e in edges]
          select = min(len(values), degree)
          edges = sorted(values, key=lambda x: -x[2])[:select]
          
          for (e, o, v) in values:
            theta[e][o] = 0.0
          for (e, o, v) in edges:
            theta[e][o] = 1.0
            
    if inplace:
      for theta in self.thetas:
        _sampling(theta)
        
      self.learn_theta(False)
      self.onehot = True
    else:
      thetas = [theta.detach().clone() for theta in self.thetas]
      for theta in thetas:
        _sampling(theta)
      return thetas
  
  def plot(self, prefix):
    thetas = self.sampling(inplace=False)
    return [plot(self.graph, theta, self.multi, prefix + name)
      for theta, name in zip(thetas, ["normal", "reduce"])]

  def log(self):
    print("Network")
    for theta in self.thetas:
      print(F.softmax(theta, dim=-1))

# Unit Test

In [14]:
import unittest

def tensor_equal(x, y):
  return (torch.sum(x == y) == x.view(-1).shape[0]).item()

class TestEdge(unittest.TestCase):
  def test_id(self):
    input = torch.randn(1, 3, 32, 32)
    operators = [nn.Identity(), None]
    model = Edge(operators, theta=torch.tensor([1.0, 0.0]))
    output = model(input)
    self.assertEqual(tensor_equal(input, output), True)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

E
ERROR: test_id (__main__.TestEdge)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-7c4fb3f2b357>", line 11, in test_id
    output = model(input)
  File "/usr/local/lib/python3.6/dist-packages/torch/nn/modules/module.py", line 722, in _call_impl
    result = self.forward(*input, **kwargs)
TypeError: forward() missing 1 required positional argument: 'theta'

----------------------------------------------------------------------
Ran 1 test in 0.003s

FAILED (errors=1)


# Training

## training

In [15]:
from tqdm.notebook import tqdm
def train(model, device, train_loader, valid_loader, optimizer, optimizerB, criterion, logger, class_array):
    model.train()
    class_array = torch.LongTensor(class_array).to(device)
    for batch_idx, (data, target) in enumerate(tqdm(train_loader)):
        data, target = data.to(device), target.to(device)

        if optimizerB:
          data_v, target_v = next(iter(valid_loader))
          data_v, target_v = data_v.to(device), target_v.to(device)

          # # theta update
          optimizerB.zero_grad()
          output = model(data_v)
          reg = torch.zeros(output.shape[0], 10).to(device)
          reg.index_add_(1, class_array, output)
          loss = criterion(reg, target_v)
          loss.backward()
          optimizerB.step()

        optimizer.zero_grad()
        output = model(data)
        reg = torch.zeros(output.shape[0], 10).to(device)
        reg.index_add_(1, class_array, output)
        loss = criterion(reg, target)
        loss.backward(retain_graph=True)
        # clip
        optimizer.step()
        
    return (None, loss.item())

In [16]:
def test(model, device, test_loader, criterion, logger, class_array):
    model.eval()
    test_loss = []
    correct = 0
    class_array = torch.tensor(class_array).to(device)
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss.append(criterion(output, target).item())
            pred = class_array[output.argmax(dim=1, keepdim=True)]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss = np.mean(np.array(test_loss))
    accuracy = 100. * correct / len(test_loader.dataset)
    
    return (None, (test_loss, accuracy))

## Utils

In [17]:
from argparse import Namespace
def dictspace(f):
  def inner(**kwds):
    return f(Namespace(**kwds))
  return inner

In [18]:
class Store():
  def __init__(self, dir="result", name="log"):
    self.dict = {}
    self.dir = dir
  
  def add(self, name, value):
    if not name in self.dict:
      self.dict[name] = []
    self.dict[name].append(value)
  
  def save(self, name="log"):
    path = os.path.join(self.dir, name + ".txt")
    with open(path, mode='w') as f:
      f.write("%s" % self.dict)

  def save_fig(self, metrix="acc", xlabel="epochs", ylabel="accuracy[%]"):
    import matplotlib.pyplot as plt
    
    times = len(self.dict[metrix])
    fig = plt.figure()
    plt.plot(np.arange(times), self.dict[metrix])
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()
    fig.savefig(os.path.join(self.dir, "%s_%d.png" % (metrix, times)))

  def __repr__(self):
    return "store in %s" % self.dict

In [19]:
def SaveModel(name, model, dir="result"):
  path = os.path.join(dir, name + ".pt")
  if not os.path.exists(dir):
    os.mkdir(dir)

  torch.save(model.state_dict(), path)
  # print(os.path.join("/content", result_path))
  # files.download(os.path.join("/content", result_path))

In [20]:
def load_dataset(train=2000, test=500, valid=0):
  #画像の変形処理
  transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
  ])

  transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
  ])

  #CIFAR-10のtrain, testsetのロード
  trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                          download=True, transform=transform)
  testset = torchvision.datasets.CIFAR10(root='./data', train=False, 
                                          download=True, transform=transform_test)
  
  trainset, validset, _ = torch.utils.data.random_split(trainset, [train, valid, 50000-train-valid])
  testset, _ = torch.utils.data.random_split(testset, [test, 10000-test])
  return trainset, testset, validset

In [21]:
class EarlyStopping:
  def __init__(self, dir="min", patent=5):
    self.list = []
    self.best = 0
    self.patent = patent
    self.count = 0
    self.order = dir == "max"

  def step(self, item):
    def _score(item):
      return item * (1 if self.order else -1)

    if len(self.list) == 0:
      self.best = _score(item)

    self.list.append(item)
    item = _score(item)
    if self.best < item:
      self.best = item
      self.count = 0
    else:
      self.count += 1

  def is_stop(self):
    return self.patent <= self.count

In [22]:
def setup_drive():
  # Install a Drive FUSE wrapper.
  # https://github.com/astrada/google-drive-ocamlfuse
  !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
  !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
  !apt-get update -qq 2>&1 > /dev/null
  !apt-get -y install -qq google-drive-ocamlfuse fuse

  # Generate auth tokens for Colab
  from google.colab import auth
  auth.authenticate_user()

  # Generate creds for the Drive FUSE library.
  from oauth2client.client import GoogleCredentials
  creds = GoogleCredentials.get_application_default()
  import getpass
  !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
  vcode = getpass.getpass()
  !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [23]:
def save_dir(dir : str, drivepath = './drive/My Drive/ml'):
  if not "getpass" in sys.modules:
    setup_drive()

  if dir:
    import subprocess
    res = subprocess.run(["cp", "-r", "./" + dir, drivepath], stdout=subprocess.PIPE)
    sys.stdout.write(res.stdout)

In [24]:
def get_time():
  import datetime
  import pytz
  dt_now = datetime.datetime.now(pytz.timezone('Asia/Tokyo'))
  return dt_now.strftime('%Y_%m_%d %H_%M %S')

In [25]:
def save_heatmap(data, path):
  import seaborn as sns
  import matplotlib.pyplot as plt
  
  plt.figure()
  sns.heatmap(data, annot=True, fmt="1.2f")
  plt.savefig(path)
  plt.close('all')

## main

In [26]:
def main(description, **kwarg):

  save_dir("")

  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda" if use_cuda else "cpu")
  print("device is %s" % device)

  # trainset, testset = load_dataset()
  kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}

  class_array = [i for i in range(10)]

  @dictspace
  def learning(args):

    store = Store(dir=args.dir)
    store.add("kwargs", args)
    theta_log = Store(dir=args.dir)

    # instantiate
    networkarg = {"depth" : args.depth, "node_num" : args.node,
                  "input" : args.input, "multi" : args.multi}
    model = CellNetwork(**networkarg)
    model.to(device)

    optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=3e-4)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, float(args.epochs), eta_min=0.001)
    optimizer_theta = optim.Adam(model.thetas, lr=args.lr_theta, betas=(0.5, 0.999), weight_decay=1e-3)
    criterion = nn.CrossEntropyLoss()

    trainset, testset, validset = load_dataset(train=args.train_size, valid=args.train_size)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, **kwargs)
    validloader = torch.utils.data.DataLoader(validset, batch_size=args.batch_size, **kwargs)
    testloader = torch.utils.data.DataLoader(testset, batch_size=args.batch_size, **kwargs)
    
    # architecture search
    time_sta = time.time()
    accuracy, loss = 0, 1e10
    model.learn_theta(False)
    !nvidia-smi
    for epoch in range(args.epochs):
      
      if epoch == args.switch:
        model.learn_theta(True)
        
      model.log()

      (_, loss_train) = train(model, device, trainloader, validloader, optimizer, optimizer_theta, criterion, None, class_array)
      (_, (loss_test, acc)) = test(model, device, testloader, criterion, None, class_array)
      scheduler.step()

      print('epoch %d, acc %s' % (epoch, acc))
      if epoch >= args.switch and epoch % 5 == 0:
        model.plot(os.path.join(args.dir, "epoch%d_" % epoch))
        if args.save_dir: save_dir(args.dir)
      store.add("loss", loss_test)
      store.add("acc", acc)
      theta_log.add("theta_n", model.thetas[0].detach().cpu().clone().numpy())
      theta_log.add("theta_r", model.thetas[1].detach().cpu().clone().numpy())
      save_heatmap(model.thetas[0].detach().cpu().clone().numpy(), os.path.join(args.dir, "theta_%s.png" % epoch))

      accuracy, loss = acc, loss_test
      if time.time() - time_sta >= 60 * args.minutes:
        break 

    print("\naccuracy ", accuracy, end=", ")
    print("loss ", loss)
    model.plot(os.path.join(args.dir, "complete_"))
    SaveModel("cell", model, dir=args.dir)

    # relearning
    stop = EarlyStopping(patent=15)
    accuracy, loss = 0, 1e10
    remodel = CellNetwork(search=False, **networkarg).to(device)
    remodel.thetas = model.sampling(inplace=False)
    remodel.sampling()
    model.cpu()
    del model
    model = remodel

    trainset, testset, _ = load_dataset(train=args.train_size * 2)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, **kwargs)
    testloader = torch.utils.data.DataLoader(testset, batch_size=args.batch_size, **kwargs)
    
    optimizer = optim.SGD(model.parameters(), lr=args.relr, momentum=args.momentum, weight_decay=3e-4)
    # scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, float(args.epochs), eta_min=0.001)
    for epoch in range(args.epochs):
      
      (_, loss_train) = train(model, device, trainloader, None, optimizer, None, criterion, None, class_array)
      (_, (loss_test, acc)) = test(model, device, testloader, criterion, None, class_array)
      # scheduler.step()

      print('epoch %d, acc %s' % (epoch, acc))
      store.add("loss", loss_test)
      store.add("acc", acc)
      store.save()
      store.save_fig()

      accuracy, loss = acc, loss_test
      stop.step(loss_test)
      if stop.is_stop():
        break

    print("\naccuracy ", accuracy, end=", ")
    print("loss ", loss)
    SaveModel("cell", model, dir=args.dir)
    print(store)
    store.save()
    theta_log.save(name="theta")
    store.save_fig()
    if args.save_dir: save_dir(args.dir)

    model.cpu()
    del model

    return loss

  kwarg['dir'] = get_time()
  if not os.path.exists(kwarg['dir']):
    os.mkdir(kwarg['dir'])
  learning(**kwarg)

In [27]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 "sec)"}'

0.150178days (12975.42sec)


In [ ]:
if __name__ == '__main__':
  # # paper
  # main("", lr=0.0250, lr_theta=0.0003, relr=0.0010, batch_size=64, train_size=20000, momentum=0.9, 
  #          epochs=50, switch=0, minutes=180, depth=8, node=7, multi=4, input=2, save_dir=True)

  # main("", lr=0.0100, lr_theta=0.0002, batch_size=64, train_size=20000, momentum=0.9, 
  #          epochs=90, switch=30, minutes=180, depth=5, node=7, multi=2)
  # main("", lr=0.0100, lr_theta=0.0002, batch_size=64, train_size=20000, momentum=0.9, 
  #          epochs=90, switch=10, minutes=180, depth=5, node=7, multi=2)
  # main("", lr=0.0050, lr_theta=0.0005, batch_size=64, train_size=4000, momentum=0.9, 
  #          epochs=15, switch=5, minutes=180, depth=4, node=4, multi=2)
  # annealing cosine scheduler no restart
  # main("", lr=0.0250, lr_theta=0.0002, relr=0.0050, batch_size=64, train_size=1000, momentum=0.9, 
  #          epochs=10, switch=0, minutes=180, depth=5, node=7, multi=4, input=2, save_dir=True)
  main("", lr=0.0200, lr_theta=0.0002, relr=0.0030, batch_size=64, train_size=25000, momentum=0.9, 
           epochs=50, switch=10, minutes=180, depth=5, node=7, multi=-1, input=2, save_dir=True)

  # main("", lr=0.0100, lr_theta=0.0002, relr=0.0010, batch_size=64, train_size=20000, momentum=0.9, 
  #          epochs=50, switch=10, minutes=180, depth=5, node=7, multi=4, input=2, save_dir=True)
  # main("", lr=0.0080, lr_theta=0.0001, relr=0.0010, batch_size=64, train_size=4000, momentum=0.9, 
  #          epochs=30, switch=10, minutes=180, depth=5, node=7, multi=4, input=2, save_dir=True)
  # main("", lr=0.0080, lr_theta=0.0001, relr=0.0010, batch_size=64, train_size=400, momentum=0.9, 
  #          epochs=1, switch=0, minutes=180, depth=3, node=7, multi=4, input=2, save_dir=True)

device is cuda
[(0, 2), (0, 3), (0, 4), (0, 5), (1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]
['none', 'skip_connect', 'max_pool_3x3', 'sep_conv_3x3', 'sep_conv_5x5', 'dil_conv_3x3', 'dil_conv_5x5']
Files already downloaded and verified
Files already downloaded and verified
Thu Sep  3 03:41:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0  

Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 240, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.6/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b


epoch 0, acc 58.2


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Network
tensor([[0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429]],
       device='cuda:0')
tensor([[0.1429, 0.1429, 0.1429, 0.1429, 0.

...


epoch 23, acc 82.0



epoch 24, acc 83.2



epoch 25, acc 83.4



epoch 26, acc 83.2


Buffered data was truncated after reaching the output size limit.

In [ ]:
def _data_transforms_cifar10(args):
  CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
  CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

  train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
  ])
  if args.cutout:
    train_transform.transforms.append(Cutout(args.cutout_length))

  valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ])
  return train_transform, valid_transform

## viz

In [ ]:
# # cause a crash to increase RAM
# [_ for _ in range(10000000000)]

In [ ]:
# def main(description, **kwarg):

#   save_dir("")

#   use_cuda = torch.cuda.is_available()
#   device = torch.device("cuda" if use_cuda else "cpu")
#   print("device is %s" % device)

#   trainset, testset = load_dataset()
#   kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

#   class_array = [i for i in range(10)]

#   @dictspace
#   def learning(args):

#     store = Store(dir=args.dir)
#     store.add("kwargs", args)
#     theta_log = Store(dir=args.dir)

#     # instantiate
#     networkarg = {"depth" : args.depth, "node_num" : args.node,
#                   "input" : args.input, "multi" : args.multi}
#     model = CellNetwork(**networkarg)
#     model.to(device)

#     optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum, weight_decay=3e-4)
#     scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, float(args.epochs), eta_min=0.001)
#     optimizer_theta = optim.Adam(model.thetas, lr=args.lr_theta, betas=(0.5, 0.999), weight_decay=1e-3)
#     criterion = nn.CrossEntropyLoss()

#     trainset, testset = load_dataset(train=args.train_size)
#     trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.batch_size, shuffle=False, **kwargs)
#     testloader = torch.utils.data.DataLoader(testset, batch_size=args.batch_size, shuffle=False, **kwargs)
    
#     # relearning
#     stop = EarlyStopping(patent=15)
#     accuracy, loss = 0, 1e10
#     remodel = CellNetwork(search=False, **networkarg).to(device)
#     remodel.thetas = [torch.tensor([
#         [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
#         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
#        device='cuda:0'), torch.tensor([
#         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
#         [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]],
#        device='cuda:0')]
#     remodel.sampling()
#     model = remodel
#     model.plot(os.path.join(args.dir, "complete_"))

#     optimizer = optim.SGD(model.parameters(), lr=args.relr, momentum=args.momentum, weight_decay=3e-4)
#     scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, float(args.epochs), eta_min=0.001)
#     for epoch in range(args.epochs):
      
#       (_, loss_train) = train(model, device, trainloader, optimizer, None, criterion, None, class_array)
#       (_, (loss_test, acc)) = test(model, device, testloader, criterion, None, class_array)
#       scheduler.step()

#       print('epoch %d, acc %s' % (epoch, acc))
#       store.add("loss", loss_test)
#       store.add("acc", acc)

#       accuracy, loss = acc, loss_test
#       stop.step(loss_test)
#       # if stop.is_stop():
#         # break

#     print("\naccuracy ", accuracy, end=", ")
#     print("loss ", loss)
#     SaveModel("cell", model, dir=args.dir)
#     print(store)
#     store.save()
#     theta_log.save(name="theta")
#     store.save_fig()
#     if args.save_dir: save_dir(args.dir)

#     del model

#     return loss

#   kwarg['dir'] = get_time()
#   if not os.path.exists(kwarg['dir']):
#     os.mkdir(kwarg['dir'])
#   learning(**kwarg)